In [1]:
# PYTHON IMPORTS
import os
import copy
from tqdm.notebook import trange, tqdm

# IMAGE IMPORTS 
from PIL import Image
import cv2

# DATA IMPORTS 
import random
import h5py
import numpy as np
import glob

# PLOTTING
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# NEURAL NETWORK
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.transforms import ToPILImage, GaussianBlur
from torchvision.transforms import Compose, RandomCrop, ToTensor, Normalize
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.models as models

# MY OWN CLASSES
from TileLocator import *
Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
data_dir = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data/"

templates_dir = f"{data_dir}templates/"
tempfiles_dir = "tempfiles/"

boundary_shapefile = f"{templates_dir}HCAD_Harris_County_Boundary.shp"
boundary_points    = f'{tempfiles_dir}boundary_points.shp'
roads_points       = f'{tempfiles_dir}roads_points.shp'

roads_fn = f"data/Roads/TexasHighways.shp"
model_checkpoint = f"{data_dir}TileLocator/071723.pth"

In [3]:
# Initialize model
model = torch.load(model_checkpoint)
model = model.to("cuda")

In [4]:
def sharpeningKernel(n_window):
    assert n_window % 2 == 1    
    kernel = np.zeros((n_window, n_window))
    
    kernel[:, n_window // 2] = -1
    kernel[n_window // 2, :] = -1
    
    kernel[n_window // 2, n_window // 2] = -1 * np.sum(kernel)
    return kernel

def resize_image(image, max_size):
    # Get the original image dimensions
    height, width = image.shape[:2]
    
    # Determine the scaling factor
    scale = max_size / max(height, width)
    
    # Calculate the new dimensions
    new_height = int(height * scale)
    new_width = int(width * scale)
    
    # Resize the image using the calculated dimensions
    resized_image = cv2.resize(image, (new_width, new_height))
    
    return resized_image


In [5]:
input_folder  = f"{data_dir}TileLocator/in/"
prep_folder   = f"{data_dir}TileLocator/in_prepped/"    
output_folder = f"{data_dir}TileLocator/out/roads/"

# Iterate over the files in the input folder
for filename in tqdm(os.listdir(input_folder)):
    # Check if the file has a supported image extension
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
        # Open the image file
        image_path = os.path.join(input_folder, filename)
        output_path = os.path.join(prep_folder, filename[:-3] + "png")
        
        image = cv2.imread(image_path, 0)
                
        image = resize_image(image, 14400)
        # image = cv2.GaussianBlur(image, (51, 51), 0)
        # image = cv2.adaptiveThreshold(image, 255, 
        #                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        #                               cv2.THRESH_BINARY, 35, 2)
        
        # Save the processed image to the output folder
        cv2.imwrite(output_path, image.astype(np.uint8))

  0%|          | 0/12 [00:00<?, ?it/s]

In [6]:
# Iterate over the files in the input folder
for filename in os.listdir(prep_folder):
    # Check if the file has a supported image extension
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
        # Open the image file
        image_path = os.path.join(prep_folder, filename)
        output_path = os.path.join(output_folder, filename[:-3] + "png")
        print(f"Processing {filename} and saved to {output_path}")
        
        # Apply the process function to the image
        processed_image = split_and_run_cnn(image_path, model, tilesize=2048)
        
        # plt.hist(processed_image.flatten())
        # plt.yscale("log")
        # plt.show()
                
        # image = np.where(processed_image > 0.1, 255, 0)
        image = processed_image * 255
        
        # Save the processed image to the output folder
        cv2.imwrite(output_path, image.astype(np.uint8))

Processing 480233IND0_0382.png and saved to C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data/TileLocator/out/roads/480233IND0_0382.png


  0%|          | 0/8 [00:00<?, ?it/s]

C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\TileLocator.py:93: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)


Processing 480296IND0_0982.png and saved to C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data/TileLocator/out/roads/480296IND0_0982.png


  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Processing 485456IND0_0982.png and saved to C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data/TileLocator/out/roads/485456IND0_0982.png


  0%|          | 0/6 [00:00<?, ?it/s]

Processing 485468IND0_0483.png and saved to C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data/TileLocator/out/roads/485468IND0_0483.png


  0%|          | 0/8 [00:00<?, ?it/s]